# Evolver Loop 7 - LB Feedback Analysis

**exp_006 (SA from scratch)**: CV=87.8112, LB=87.8112 (gap: 0.0000)

The SA improved from 87.99 to 87.81 (0.18 points) by optimizing only N=2-4.

## Key Insights from Top Kernels

1. **Jonathan Chan's kernel** shows the winning approach:
   - Ensemble from 15+ sources (best per-N)
   - SA with T0=1.0, T_min=0.000005
   - **Fractional translation** with step sizes: 0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001
   - N=1 is FIXED at optimal: x=0, y=0, deg=45

2. **The gap analysis**:
   - Current: 87.81 (Zaburo + SA on N=2-4)
   - Pre-optimized baseline: 70.6 (but has overlaps)
   - Target: 68.89
   - Gap: 18.9 points

In [1]:
import pandas as pd
import numpy as np
import json

# Load the pre-optimized baseline per-N scores
baseline_path = '/home/nonroot/snapshots/santa-2025/21329067673/submission/submission.csv'
zaburo_path = '/home/code/experiments/005_zaburo_rowbased/submission.csv'
sa_path = '/home/code/experiments/006_sa_from_scratch/submission.csv'

# Load metrics from SA experiment
with open('/home/code/experiments/006_sa_from_scratch/metrics.json', 'r') as f:
    sa_metrics = json.load(f)

print("SA Experiment Results:")
print(f"Initial score (Zaburo): {sa_metrics['initial_score']:.6f}")
print(f"Final score (SA): {sa_metrics['cv_score']:.6f}")
print(f"Improvement: {sa_metrics['improvement']:.6f}")

SA Experiment Results:
Initial score (Zaburo): 87.991248
Final score (SA): 87.811181
Improvement: 0.180067


In [2]:
# Compare per-N scores: SA vs Zaburo
sa_per_n = sa_metrics['per_n_scores']

print("\nPer-N Score Comparison (SA vs Zaburo):")
print("="*60)

# Top 20 N values by score (these contribute most to total)
sorted_n = sorted(sa_per_n.items(), key=lambda x: x[1], reverse=True)[:20]
for n_str, score in sorted_n:
    n = int(n_str)
    print(f"N={n:3d}: score={score:.6f}")


Per-N Score Comparison (SA vs Zaburo):
N=  5: score=0.800000
N=  4: score=0.724729
N=  6: score=0.666667
N=  1: score=0.661250
N=  2: score=0.655506
N=  7: score=0.630000
N= 13: score=0.603077
N= 15: score=0.600000
N=  3: score=0.578656
N= 16: score=0.562500
N= 14: score=0.560000
N=  8: score=0.551250
N= 11: score=0.545682
N=  9: score=0.537778
N= 17: score=0.529412
N= 28: score=0.529375
N= 19: score=0.522237
N= 31: score=0.516129
N= 29: score=0.511121
N= 12: score=0.500208


In [3]:
# Calculate theoretical minimum for small N
# N=1: optimal is 45 degrees, side = 0.813173, score = 0.661250
# This is already optimal in baseline

print("\nTheoretical Analysis:")
print("="*60)
print("N=1: Optimal at 45°, score = 0.661250 (ALREADY OPTIMAL)")
print("N=2-4: SA improved these")
print("N=5-10: SA showed NO improvement despite many accepted moves")
print("")
print("PROBLEM: SA move sizes are too coarse!")
print("- Our SA uses: ±0.1 translation, ±10° rotation")
print("- Top kernels use: ±0.00001 to ±0.001 translation (100-10000x finer!)")


Theoretical Analysis:
N=1: Optimal at 45°, score = 0.661250 (ALREADY OPTIMAL)
N=2-4: SA improved these
N=5-10: SA showed NO improvement despite many accepted moves

PROBLEM: SA move sizes are too coarse!
- Our SA uses: ±0.1 translation, ±10° rotation
- Top kernels use: ±0.00001 to ±0.001 translation (100-10000x finer!)


In [4]:
# The key insight: We need to use the PRE-OPTIMIZED baseline as starting point
# It has better per-N scores, we just need to fix the overlapping N values

print("\nSTRATEGY:")
print("="*60)
print("1. Load pre-optimized baseline (score ~70.6)")
print("2. Identify N values with overlaps (29 N values)")
print("3. For overlapping N values, use Zaburo's valid solution")
print("4. Apply fractional translation refinement to ALL N values")
print("5. This should give us a valid submission with score ~72-75")
print("")
print("Then iterate:")
print("- Apply SA with MUCH finer moves (0.001 to 0.00001)")
print("- Keep best per-N across all experiments")
print("- Target: 68.89")


STRATEGY:
1. Load pre-optimized baseline (score ~70.6)
2. Identify N values with overlaps (29 N values)
3. For overlapping N values, use Zaburo's valid solution
4. Apply fractional translation refinement to ALL N values
5. This should give us a valid submission with score ~72-75

Then iterate:
- Apply SA with MUCH finer moves (0.001 to 0.00001)
- Keep best per-N across all experiments
- Target: 68.89


In [5]:
# Check which N values have overlaps in the pre-optimized baseline
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon

getcontext().prec = 25
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self.polygon = self._create_polygon()
    
    def _create_polygon(self):
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        return affinity.translate(rotated,
                                  xoff=float(self.center_x * scale_factor),
                                  yoff=float(self.center_y * scale_factor))

def check_overlap(trees):
    polygons = [t.polygon for t in trees]
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return True
    return False

print("ChristmasTree class defined")

ChristmasTree class defined


In [6]:
# Load pre-optimized baseline and check overlaps
def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_submission(path):
    df = pd.read_csv(path)
    trees_by_n = {}
    for _, row in df.iterrows():
        id_parts = row['id'].split('_')
        n = int(id_parts[0])
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        if n not in trees_by_n:
            trees_by_n[n] = []
        trees_by_n[n].append(ChristmasTree(center_x=x, center_y=y, angle=deg))
    return trees_by_n

print("Loading pre-optimized baseline...")
baseline_trees = load_submission(baseline_path)
print(f"Loaded {len(baseline_trees)} N values")

# Check for overlaps
overlapping_n = []
for n in range(1, 201):
    if check_overlap(baseline_trees[n]):
        overlapping_n.append(n)

print(f"\nN values with overlaps: {len(overlapping_n)}")
print(f"Overlapping N: {overlapping_n[:20]}...")

Loading pre-optimized baseline...


Loaded 200 N values



N values with overlaps: 3
Overlapping N: [30, 134, 166]...


In [7]:
# Calculate baseline per-N scores
from shapely.ops import unary_union

def calculate_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(str(bounds[0])) / scale_factor
    miny = Decimal(str(bounds[1])) / scale_factor
    maxx = Decimal(str(bounds[2])) / scale_factor
    maxy = Decimal(str(bounds[3])) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return float(max(width, height))

def calculate_score(trees, n):
    side = calculate_side_length(trees)
    return (side ** 2) / n

# Calculate baseline scores for valid N values
baseline_scores = {}
for n in range(1, 201):
    if n not in overlapping_n:
        baseline_scores[n] = calculate_score(baseline_trees[n], n)

print(f"\nBaseline scores for {len(baseline_scores)} valid N values:")
print(f"Total (valid only): {sum(baseline_scores.values()):.6f}")
print(f"\nTop 10 valid N by score:")
sorted_valid = sorted(baseline_scores.items(), key=lambda x: x[1], reverse=True)[:10]
for n, score in sorted_valid:
    print(f"N={n:3d}: score={score:.6f}")


Baseline scores for 197 valid N values:
Total (valid only): 69.573570

Top 10 valid N by score:
N=  1: score=0.661250
N=  2: score=0.450779
N=  3: score=0.434745
N=  5: score=0.416850
N=  4: score=0.416545
N=  7: score=0.399897
N=  6: score=0.399610
N=  9: score=0.387415
N=  8: score=0.385407
N= 15: score=0.376949


In [8]:
# Compare: Baseline (valid N) vs SA (all N)
print("\nComparison: Baseline vs SA for valid N values")
print("="*60)

improvements = []
for n in range(1, 201):
    if n in baseline_scores:
        sa_score = sa_per_n[str(n)]
        base_score = baseline_scores[n]
        diff = sa_score - base_score  # positive = SA is worse
        if abs(diff) > 0.0001:
            improvements.append((n, base_score, sa_score, diff))

print(f"\nN values where baseline is BETTER than SA:")
for n, base, sa, diff in sorted(improvements, key=lambda x: x[3], reverse=True)[:15]:
    print(f"N={n:3d}: baseline={base:.6f}, SA={sa:.6f}, diff={diff:.6f}")


Comparison: Baseline vs SA for valid N values

N values where baseline is BETTER than SA:
N=  5: baseline=0.416850, SA=0.800000, diff=0.383150
N=  4: baseline=0.416545, SA=0.724729, diff=0.308184
N=  6: baseline=0.399610, SA=0.666667, diff=0.267056
N= 13: baseline=0.372294, SA=0.603077, diff=0.230783
N=  7: baseline=0.399897, SA=0.630000, diff=0.230103
N= 15: baseline=0.376949, SA=0.600000, diff=0.223051
N=  2: baseline=0.450779, SA=0.655506, diff=0.204727
N= 14: baseline=0.369543, SA=0.560000, diff=0.190457
N= 16: baseline=0.374128, SA=0.562500, diff=0.188372
N= 11: baseline=0.374924, SA=0.545682, diff=0.170758
N=  8: baseline=0.385407, SA=0.551250, diff=0.165843
N= 28: baseline=0.366105, SA=0.529375, diff=0.163270
N= 17: baseline=0.370040, SA=0.529412, diff=0.159371
N= 19: baseline=0.368615, SA=0.522237, diff=0.153622
N=  9: baseline=0.387415, SA=0.537778, diff=0.150363


In [9]:
# KEY INSIGHT: The pre-optimized baseline has MUCH better scores for most N values
# We just need to fix the overlapping ones

print("\n" + "="*60)
print("KEY INSIGHT")
print("="*60)
print("")
print("The pre-optimized baseline has MUCH better per-N scores!")
print("But it has overlaps in some N values.")
print("")
print("STRATEGY:")
print("1. Use baseline for N values WITHOUT overlaps")
print("2. Use Zaburo/SA for N values WITH overlaps")
print("3. Apply fractional translation to improve further")
print("")
print(f"Baseline valid N count: {len(baseline_scores)}")
print(f"Overlapping N count: {len(overlapping_n)}")
print(f"")
print(f"Estimated hybrid score:")
print(f"  Baseline valid: {sum(baseline_scores.values()):.6f}")
print(f"  SA for overlapping: {sum(sa_per_n[str(n)] for n in overlapping_n):.6f}")
print(f"  Total: {sum(baseline_scores.values()) + sum(sa_per_n[str(n)] for n in overlapping_n):.6f}")


KEY INSIGHT

The pre-optimized baseline has MUCH better per-N scores!
But it has overlaps in some N values.

STRATEGY:
1. Use baseline for N values WITHOUT overlaps
2. Use Zaburo/SA for N values WITH overlaps
3. Apply fractional translation to improve further

Baseline valid N count: 197
Overlapping N count: 3

Estimated hybrid score:
  Baseline valid: 69.573570
  SA for overlapping: 1.302297
  Total: 70.875867
